In [14]:
import torch
from torch import nn 
import math 
import torch.nn.functional as F

In [17]:
def scaled_dot_product(q,k,v, mask=None):
    # q, k ,v = 30 x 8 x 200 x 64
    d_k = q.size()[-1] # 64
    scaled = torch.matmul(q,k.transpose(-1,-2)) / math.sqrt(d_k) # 30 x 8 x 200 x 200
    if mask is not None:
        scaled += mask # mask: (200 x 200) broadcasting add
    attention = F.softmax(scaled, dim=-1) # 30 x 8 x 200 x 200
    values = torch.matmul(attention,v) # 30 x 8 x 200 x 64
    return values,attention

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model # 512
        self.num_heads = num_heads # 8 
        self.head_dim = d_model // num_heads # 64
        self.qkv_layer = nn.Linear(d_model, 3 * d_model) # 512 x 1536
        self.linear_layer = nn.Linear(d_model, d_model) # 512 x 512
    
    def forward(self, x, mask=None):
        batch_size, sequence_length, d_model = x.size() # 30 x 200 x 512
        qkv = self.qkv_layer(x) # 30 x 200 x (1536)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim) # 30 x 200 x 8 x 192
        qkv = qkv.permute(0,2,1,3) # 30 x 8 x 200 x 192
        q,k,v = qkv.chunk(3, dim=-1) # each 30 x 8 x 200 x 64
        values, attention = scaled_dot_product(q, k, v) ## Values: 30 x 8 x 200 x 64, Attention: 30 x 8 x 200 x 200
        values = values.reshape(batch_size,sequence_length, self.num_heads*self.head_dim) # 30 x 200 x 512
        out = self.linear_layer(values) # 30 x 200 x 512
        return out

class MultiHeadCrossAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model) # 1024
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, y, mask=None):
        batch_size, sequence_length, d_model = x.size() # 30 x 200 x 512
        kv = self.kv_layer(x) # 30 x 200 x 1024
        q = self.q_layer(y) # 30 x 200 x 512
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)  # 30 x 200 x 8 x 128
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)  # 30 x 200 x 8 x 64
        kv = kv.permute(0, 2, 1, 3) # 30 x 8 x 200 x 128
        q = q.permute(0, 2, 1, 3) # 30 x 8 x 200 x 64
        k, v = kv.chunk(2, dim=-1) # K: 30 x 8 x 200 x 64, v: 30 x 8 x 200 x 64
        values, attention = scaled_dot_product(q, k, v, mask) #  30 x 8 x 200 x 64
        values = values.reshape(batch_size, sequence_length, d_model) #  30 x 200 x 512
        out = self.linear_layer(values)  #  30 x 200 x 512
        return out  #  30 x 200 x 512
    
    
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape # [512]
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # [512] stddev of values
        self.beta = nn.Parameter(torch.zeros(parameters_shape)) # [512] mean of values
        
    
    def forward(self, inputs): # 30 x 200 x 512
        dims = [(-i + 1) for i in range(len(self.parameters_shape))] # [-1]
        mean = inputs.mean(dim=dims, keepdim=True) # 30 x 200 x 1
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True) # 30 x 200 x 1
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std # 30 x 200 x 512
        out = self.gamma * y + self.beta # 30 x 200 x 512
        return out
    
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden) # 512 x 2048
        self.linear2 = nn.Linear(hidden, d_model) # 2048 x 512
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)
        
    def forward(self, x): # 30 x 200 x 512
        x = self.linear1(x) #  30 x 200 x 2048
        x = self.relu(x) # 30 x 200 x 2048
        x = self.dropout(x) # 30 x 200 x 2048
        x = self.linear2(x) # 30 x 200 x 512
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)
    
    def forward(self, x, y, decoder_mask):
        _y = y
        y = self.self_attention(y, mask = decoder_mask)
        y = self.dropout1(y)
        y = self.norm1(y + _y)
        
        _y = y
        y = self.encoder_decoder_attention(x, y, mask=None)
        y = self.dropout2(y)
        y = self.norm2(y + _y)
        
        _y = y  
        y = self.ffn(y) 
        y = self.dropout3(y) 
        y = self.norm3(y + _y) 
        return y # Every output essentially gives #30 x 200 x 512
        
        
        
class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, mask = inputs
        for module in self._modules.values():
            y = module(x, y, mask)
        return y

class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers=1):
        super().__init__()
        self.layers = SequentialDecoder(*[DecoderLayer(d_model,ffn_hidden,num_heads,drop_prob)
                                     for _ in range(num_layers)])
    def forward(self, x, y, mask):
        # x : 30 x 200 x 512
        # y : 30 x 200 x 512
        # mask : 200 x 200
        y = self.layers(x, y, mask)
        return y

In [18]:
d_model = 512
num_heads = 8 
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048 # feed forward network : to learn additional information
num_layers = 5

x = torch.randn( (batch_size, max_sequence_length, d_model) )
y = torch.randn( (batch_size, max_sequence_length, d_model) )
mask = torch.full([max_sequence_length, max_sequence_length], float('-inf'))
mask = torch.triu(mask, diagonal=1)
decoder = Decoder (d_model, ffn_hidden, num_heads, drop_prob, num_layers)
out = decoder(x, y, mask)

In [19]:
out

tensor([[[ 2.1942, -0.4041,  0.6649,  ...,  0.4128,  1.4645,  0.5077],
         [ 0.0546,  0.4517, -0.2764,  ..., -0.8424, -0.7779,  0.3019],
         [ 0.5436, -0.0474,  1.4237,  ..., -0.7775,  2.2516, -0.7642],
         ...,
         [-0.5175, -1.7388, -0.5346,  ..., -0.0539, -0.7528,  1.0837],
         [-1.8288, -0.2687,  0.6401,  ..., -0.2036,  0.1303, -0.4568],
         [ 1.5989, -0.4179, -0.2042,  ...,  0.4671,  0.7234,  0.8025]],

        [[ 0.3220, -0.6477,  1.4009,  ...,  0.9834, -0.8807,  2.7400],
         [ 0.4366, -0.6596, -0.7743,  ..., -0.1330,  0.1245, -0.3545],
         [ 1.5920,  1.6845, -0.8578,  ...,  0.5049,  0.1056,  1.9454],
         ...,
         [ 0.1418, -0.7464,  0.0419,  ...,  0.9089, -1.2429, -0.4445],
         [ 1.3504, -1.1851,  1.7925,  ..., -0.5700,  1.6587, -0.6142],
         [ 0.8054,  0.2225, -1.7680,  ...,  1.4677, -1.3800, -0.8785]],

        [[-0.1670, -0.4030, -0.3678,  ...,  2.4876,  0.3836,  1.5137],
         [ 0.0804,  0.2313, -0.7121,  ..., -0